# Common functions
Contains common functions needed for the script snippets below

In [1]:
from pyspark.sql.functions import when, to_date, avg, to_timestamp, col, cast
from pyspark.sql.dataframe import DataFrame

storageAccountName = "dynamicsstagingsa.dfs.core.windows.net"
containerName = "stagingdata"
manifestPath = "%s/sample/CDS/model.json" % (containerName)
outputPath = "output"

def readEntityFromLake(storageAccount, manifest, entityName):
    dataFrame = (spark.read.format("com.microsoft.cdm")
        .option("storage", storageAccount)
        .option("manifestPath", manifest)
        .option("entity", entityName)
        .option("mode", "permissive")
        .load())

    return dataFrame

def writeToCsv(dataFrame: DataFrame, csvName):
    csvPath = 'abfss://%s@%s/%s/%s' % (containerName, storageAccountName, outputPath, csvName)
    dataFrame.write.csv(csvPath, mode = 'overwrite', header = 'true')


StatementMeta(SparkPool01, 4, 2, Finished, Available)

# Cases per day
This calculates the number of cases per day, by queue.

In [2]:
queueDf = readEntityFromLake(storageAccountName, manifestPath, "queue")
queueItemsDf = readEntityFromLake(storageAccountName, manifestPath, "queueitem")
incidentDf = readEntityFromLake(storageAccountName, manifestPath, "incident")

# Define columns to select
incidentColumns = ["createdon", "incidentid", "title"]
queueItemsColumns = ["queueid", "objectid"]
queueColumns = ["queueid", "name"]
joinedDfColumns = ["incidentid", "createdon", "name"]

# Filter data frames for required rows and columns
filteredIncidentDf = incidentDf \
                        .filter(incidentDf.createdon.isNotNull()) \
                        .select(*incidentColumns)

filteredQueueItemDf = queueItemsDf \
                        .filter(queueItemsDf.queueid.isNotNull() & queueItemsDf.objectid.isNotNull()) \
                        .select(*queueItemsColumns) \
                        .withColumnRenamed("queueid", "qi_queueid")

filteredQueueDf = queueDf \
                    .withColumn("name", when((queueDf.name.isNull()) | (queueDf.name == ""), "<Unnamed Queue>")
                                        .otherwise(queueDf.name)) \
                    .select(*queueColumns)

# Join the data sets
joinedDf = filteredQueueItemDf \
                .join(filteredQueueDf, filteredQueueItemDf.qi_queueid == filteredQueueDf.queueid, "inner") \
                .join(filteredIncidentDf, filteredQueueItemDf.objectid == filteredIncidentDf.incidentid, "rightouter") \
                .select(*joinedDfColumns)

joinedDf = joinedDf \
                .withColumn("name", when(joinedDf.name.isNull(), "<No_Queue_Assigned>")
                                    .otherwise(joinedDf.name)) \
                .withColumn("createdon_date", to_date(joinedDf.createdon))

# Group joined data set on created date and queue name
groupedDf = joinedDf \
                .groupBy(joinedDf.createdon_date, joinedDf.name) \
                .count() \
                .orderBy(joinedDf.createdon_date, joinedDf.name) \
                .withColumnRenamed("createdon_date", "date") \
                .withColumnRenamed("name", "queue_name")

groupedDf.show(truncate=False)
writeToCsv(groupedDf, "IncidentsPerDay")

StatementMeta(SparkPool01, 4, 3, Finished, Available)

+----------+----------------------+-----+
|date      |queue_name            |count|
+----------+----------------------+-----+
|2021-12-23|<No_Queue_Assigned>   |1    |
|2022-11-01|Forecast case queue 01|81   |
|2022-11-01|Forecast case queue 02|109  |
|2022-11-02|Forecast case queue 01|94   |
|2022-11-02|Forecast case queue 02|127  |
|2022-11-03|Forecast case queue 01|90   |
|2022-11-03|Forecast case queue 02|121  |
|2022-11-04|Forecast case queue 01|90   |
|2022-11-04|Forecast case queue 02|120  |
|2022-11-05|Forecast case queue 01|93   |
|2022-11-05|Forecast case queue 02|127  |
|2022-11-07|Forecast case queue 01|71   |
|2022-11-07|Forecast case queue 02|90   |
|2022-11-08|Forecast case queue 01|99   |
|2022-11-08|Forecast case queue 02|120  |
|2022-11-09|Forecast case queue 01|100  |
|2022-11-09|Forecast case queue 02|120  |
|2022-11-10|Forecast case queue 01|100  |
|2022-11-10|Forecast case queue 02|120  |
|2022-11-11|Forecast case queue 01|100  |
+----------+----------------------

# Participants by session
This calculates the number of participants by session

In [3]:
sessionParticipantDf = readEntityFromLake(storageAccountName, manifestPath, "msdyn_sessionparticipant")

# Define columns to select
sessionParticipantColumns = ["createdon","msdyn_omnichannelsession", "msdyn_omnichannelsessionname", "msdyn_sessionparticipantid"]

# Filter data frames for required rows and columns
filteredSessionDf = sessionParticipantDf \
                        .filter(sessionParticipantDf.createdon.isNotNull()) \
                        .select(*sessionParticipantColumns) \
                        .withColumn("createdon_date", to_date(sessionParticipantDf.createdon))

# Group joined data set on created date and queue name
groupedDf = filteredSessionDf \
                .groupBy(filteredSessionDf.createdon_date,filteredSessionDf.msdyn_omnichannelsession, filteredSessionDf.msdyn_omnichannelsessionname) \
                .count() \
                .orderBy(filteredSessionDf.createdon_date, filteredSessionDf.msdyn_omnichannelsession) \
                .withColumnRenamed("createdon_date", "date")

groupedDf.show(truncate=False)
writeToCsv(groupedDf, "ParticipantsPerSession")

StatementMeta(SparkPool01, 4, 4, Finished, Available)

+----------+------------------------------------+---------------------------------------------+-----+
|date      |msdyn_omnichannelsession            |msdyn_omnichannelsessionname                 |count|
+----------+------------------------------------+---------------------------------------------+-----+
|2023-03-07|000f4df8-42bd-ed11-b594-000d3a5d2dc9|Session for Forecast demodata chat 41457b2e5a|1    |
|2023-03-07|003ae761-42bd-ed11-b594-000d3a5d2dc9|Session for Forecast demodata chat 235c2b277b|1    |
|2023-03-07|00be383a-43bd-ed11-b594-000d3a5d2dc9|Session for Forecast demodata chat 5c3a78eea2|1    |
|2023-03-07|010a26f8-41bd-ed11-b594-000d3a5d2dc9|Session for Forecast demodata chat 45c83e1289|1    |
|2023-03-07|01332b70-43bd-ed11-b594-000d3a5d2dc9|Session for Forecast demodata chat af0a336cce|1    |
|2023-03-07|013a35b5-41bd-ed11-b594-000d3a5d2dc9|Session for Forecast demodata chat d7bb1c339f|1    |
|2023-03-07|014f3034-43bd-ed11-b594-000d3a5d2dc9|Session for Forecast demodata cha

# Average Scheduled duration, actual duration and on hold time for sessions

In [4]:
sessionDf = readEntityFromLake(storageAccountName, manifestPath, "msdyn_ocsession")

# Define columns to select
sessionColumns = ["createdon","msdyn_sessionid", "actualdurationminutes", "scheduleddurationminutes","onholdtime"]

# Filter data frames for required rows and columns
filteredSessionDf = sessionDf \
                        .filter(sessionDf.createdon.isNotNull()) \
                        .select(*sessionColumns) \
                        .withColumn("createdon_date", to_date(sessionDf.createdon))

# Group joined data set on created date and queue name
groupedDf = filteredSessionDf.groupBy(filteredSessionDf.createdon_date) \
                .agg(avg(filteredSessionDf.actualdurationminutes),avg(filteredSessionDf.scheduleddurationminutes),avg(filteredSessionDf.onholdtime)) \
                .orderBy(filteredSessionDf.createdon_date) \
                .withColumnRenamed("createdon_date", "date")

groupedDf.show(truncate=False)
writeToCsv(groupedDf, "SessionMetrics")

StatementMeta(SparkPool01, 4, 5, Finished, Available)

+----------+--------------------------+-----------------------------+---------------+
|date      |avg(actualdurationminutes)|avg(scheduleddurationminutes)|avg(onholdtime)|
+----------+--------------------------+-----------------------------+---------------+
|2022-12-01|null                      |null                         |null           |
|2022-12-02|null                      |null                         |null           |
|2022-12-03|null                      |null                         |null           |
|2022-12-04|null                      |null                         |null           |
|2022-12-05|null                      |null                         |null           |
|2022-12-06|null                      |null                         |null           |
|2022-12-07|null                      |null                         |null           |
|2022-12-08|null                      |null                         |null           |
|2022-12-09|null                      |null           

# Time to assign live work item

In [5]:
workItemDf = readEntityFromLake(storageAccountName, manifestPath, "msdyn_ocliveworkitem")

# Define columns to select
workItemColumns = ["createdon","msdyn_activeagentassignedon"]

# Filter data frames for required rows and columns
filteredDf = workItemDf \
                        .filter(workItemDf.createdon.isNotNull()) \
                        .select(*workItemColumns) \
                        .withColumn("createdon_date", to_date(workItemDf.createdon)) \
                        .withColumn("createdon_timestamp", to_timestamp(workItemDf.createdon)) \
                        .withColumn("assignedon_timestamp", to_timestamp(workItemDf.msdyn_activeagentassignedon)) \
                        .withColumn("timetoassigninseconds", col("assignedon_timestamp").cast("long") - col("createdon_timestamp").cast("long"))


# Group joined data set on created date and queue name
groupedDf = filteredDf.groupBy(filteredDf.createdon_date) \
                .agg(avg(filteredDf.timetoassigninseconds)) \
                .orderBy(filteredDf.createdon_date) \
                .withColumnRenamed("createdon_date", "date")

groupedDf.show(truncate=False)
writeToCsv(groupedDf, "TimeToAssignWorkItem")

StatementMeta(SparkPool01, 4, 6, Finished, Available)

+----------+--------------------------+
|date      |avg(timetoassigninseconds)|
+----------+--------------------------+
|2022-12-01|3.887218045112782         |
|2022-12-02|3.849462365591398         |
|2022-12-03|3.972375690607735         |
|2022-12-04|4.049180327868853         |
|2022-12-05|4.0701754385964914        |
|2022-12-06|4.046632124352332         |
|2022-12-07|3.9244186046511627        |
|2022-12-08|3.7559808612440193        |
|2022-12-09|4.048648648648649         |
|2022-12-10|4.052910052910053         |
|2022-12-11|3.955223880597015         |
|2022-12-12|4.037593984962406         |
|2022-12-13|3.9887640449438204        |
|2022-12-14|4.043478260869565         |
|2022-12-15|4.031413612565445         |
|2022-12-16|3.8944723618090453        |
|2022-12-17|4.0046728971962615        |
|2022-12-18|3.8688524590163933        |
|2022-12-19|3.852713178294574         |
|2022-12-20|3.9408602150537635        |
+----------+--------------------------+
only showing top 20 rows



# Average wrap up time

In [6]:
workItemDf = readEntityFromLake(storageAccountName, manifestPath, "msdyn_ocliveworkitem")
# Define columns to select
workItemColumns = ["createdon","msdyn_activeagentassignedon","msdyn_wrapupinitiatedon","actualend"]
# Filter data frames for required rows and columns
filteredDf = workItemDf \
                        .filter(workItemDf.createdon.isNotNull()) \
                        .select(*workItemColumns) \
                        .withColumn("createdon_date", to_date(workItemDf.createdon)) \
                        .withColumn("wrapupstart_timestamp", to_timestamp(workItemDf.msdyn_wrapupinitiatedon)) \
                        .withColumn("end_timestamp", to_timestamp(workItemDf.actualend)) \
                        .withColumn("timetowrapinseconds", col("actualend").cast("long") - col("msdyn_wrapupinitiatedon").cast("long"))

# Group joined data set on created date and queue name
groupedDf = filteredDf.groupBy(filteredDf.createdon_date) \
                .agg(avg(filteredDf.timetowrapinseconds)) \
                .orderBy(filteredDf.createdon_date) \
                .withColumnRenamed("createdon_date", "date")
groupedDf.show(truncate=False)
writeToCsv(groupedDf, "WrapUpTime")

StatementMeta(SparkPool01, 4, 7, Finished, Available)

+----------+------------------------+
|date      |avg(timetowrapinseconds)|
+----------+------------------------+
|2022-12-01|74.48872180451127       |
|2022-12-02|75.18279569892474       |
|2022-12-03|74.41988950276243       |
|2022-12-04|76.63934426229508       |
|2022-12-05|74.25438596491227       |
|2022-12-06|73.93782383419689       |
|2022-12-07|74.27325581395348       |
|2022-12-08|75.49282296650718       |
|2022-12-09|74.63783783783784       |
|2022-12-10|75.56613756613757       |
|2022-12-11|76.08955223880596       |
|2022-12-12|75.25563909774436       |
|2022-12-13|75.80337078651685       |
|2022-12-14|74.45108695652173       |
|2022-12-15|75.31937172774869       |
|2022-12-16|75.04020100502512       |
|2022-12-17|74.8644859813084        |
|2022-12-18|74.73770491803279       |
|2022-12-19|75.11627906976744       |
|2022-12-20|73.95161290322581       |
+----------+------------------------+
only showing top 20 rows



# Average idle time for agent in a session

In [7]:
sesionParticipantDf = readEntityFromLake(storageAccountName, manifestPath, "msdyn_sessionparticipant")
# Define columns to select
workItemColumns = ["createdon","msdyn_idletime","msdyn_agentidname"]
# Filter data frames for required rows and columns
filteredSessionDf = sesionParticipantDf \
                        .filter(sesionParticipantDf.createdon.isNotNull()) \
                        .select(*workItemColumns) \
                        .withColumn("createdon_date", to_date(sesionParticipantDf.createdon)) \
# Group joined data set on created date and queue name
groupedDf = filteredSessionDf.groupBy(filteredSessionDf.createdon_date,filteredSessionDf.msdyn_agentidname) \
                .agg(avg(filteredSessionDf.msdyn_idletime)) \
                .orderBy(filteredSessionDf.createdon_date) \
                .withColumnRenamed("createdon_date", "date")
groupedDf.show(truncate=False)
writeToCsv(groupedDf, "IdleTimeSession")

StatementMeta(SparkPool01, 4, 8, Finished, Available)

+----------+-----------------+-------------------+
|date      |msdyn_agentidname|avg(msdyn_idletime)|
+----------+-----------------+-------------------+
|2023-03-07|aurora user13    |0.0                |
|2023-03-07|aurora user20    |0.0                |
|2023-03-07|aurora user05    |0.0                |
|2023-03-07|aurora user08    |0.0                |
|2023-03-07|aurora user06    |0.0                |
|2023-03-07|aurora user07    |0.0                |
|2023-03-07|aurora user02    |0.0                |
|2023-03-07|aurora user11    |0.0                |
|2023-03-07|aurora user10    |0.0                |
|2023-03-07|aurora user09    |0.0                |
|2023-03-07|aurora user18    |0.0                |
|2023-03-07|aurora user14    |0.0                |
|2023-03-07|aurora user03    |0.0                |
|2023-03-07|aurora user17    |0.0                |
|2023-03-07|aurora user16    |0.0                |
|2023-03-07|aurora user04    |0.0                |
|2023-03-07|aurora user19    |0

# Average session duration

In [8]:
sessionDf = readEntityFromLake(storageAccountName, manifestPath, "msdyn_ocsession")
# Define columns to select
workItemColumns = ["createdon","actualstart","actualend"]
# Filter data frames for required rows and columns
filteredDf = sessionDf \
                        .filter(sessionDf.createdon.isNotNull()) \
                        .select(*workItemColumns) \
                        .withColumn("createdon_date", to_date(sessionDf.createdon)) \
                        .withColumn("start_timestamp", to_timestamp(sessionDf.actualstart)) \
                        .withColumn("end_timestamp", to_timestamp(sessionDf.actualend)) \
                        .withColumn("timeinseconds", col("actualend").cast("long") - col("start_timestamp").cast("long"))

# Group joined data set on created date and queue name
groupedDf = filteredDf.groupBy(filteredDf.createdon_date) \
                .agg(avg(filteredDf.timeinseconds)) \
                .orderBy(filteredDf.createdon_date) \
                .withColumnRenamed("createdon_date", "date")
groupedDf.show(truncate=False)
writeToCsv(groupedDf, "SessionDuration")

StatementMeta(SparkPool01, 4, 9, Finished, Available)

+----------+------------------+
|date      |avg(timeinseconds)|
+----------+------------------+
|2022-12-01|null              |
|2022-12-02|null              |
|2022-12-03|null              |
|2022-12-04|null              |
|2022-12-05|null              |
|2022-12-06|null              |
|2022-12-07|null              |
|2022-12-08|null              |
|2022-12-09|null              |
|2022-12-10|null              |
|2022-12-11|null              |
|2022-12-12|null              |
|2022-12-13|null              |
|2022-12-14|null              |
|2022-12-15|null              |
|2022-12-16|null              |
|2022-12-17|null              |
|2022-12-18|null              |
|2022-12-19|null              |
|2022-12-20|null              |
+----------+------------------+
only showing top 20 rows

